<a href="https://colab.research.google.com/github/kvdprasad89ai/machine-learning/blob/master/ReadingtextFromImageUsingCV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import pandas as pd
import requests
import io
import json
import os
from google.colab import drive

In [2]:
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

Files in '/content': ['.config', 'drive', 'sample_data']


In [4]:
folder_path = "/content/drive/My Drive/OCR_image_text_detection/data/"
def load_images_from_folder(folder_path):
    image_dic = {}
    images = []
    for filename in os.listdir(folder_path):
        print(filename)
        img = cv2.imread(os.path.join(folder_path,filename))
        if img is not None:
            images.append(img)
            image_dic[filename] = img
    return images,image_dic
images_list,image_dic = load_images_from_folder(folder_path)

Time series.PNG
image data.PNG
chat bots.PNG
chat bot 2.PNG
NLP 3.PNG
Chat Bods Intro.PNG
NLP2.PNG
NLP with Chotbot.PNG


In [5]:
def get_image_info(image_dic):
  for key in image_dic:
    image_name = image_dic[key]
    height, width, _ = image_name.shape
    print("Image Name:{},Image Height:{}, Image Width:{}".format(key,height, width))

get_image_info(image_dic)

Image Name:Time series.PNG,Image Height:820, Image Width:1009
Image Name:image data.PNG,Image Height:801, Image Width:924
Image Name:chat bots.PNG,Image Height:818, Image Width:945
Image Name:chat bot 2.PNG,Image Height:801, Image Width:941
Image Name:NLP 3.PNG,Image Height:749, Image Width:1127
Image Name:Chat Bods Intro.PNG,Image Height:781, Image Width:677
Image Name:NLP2.PNG,Image Height:780, Image Width:837
Image Name:NLP with Chotbot.PNG,Image Height:838, Image Width:790


In [6]:
# Ocr
url_api = "https://api.ocr.space/parse/image"


In [7]:
text_dict = {}
def getText_From_Image(image_dic):
  for key in image_dic:
    image_name = image_dic[key]
    _, compressedimage = cv2.imencode(".PNG", image_name, [1, 90])
    file_bytes = io.BytesIO(compressedimage)
    result = requests.post(url_api,
              files = {key: file_bytes},
              data = {"apikey": "ae902d73cd88957",
                      "language": "eng"})
    
    result = result.content.decode()
    result = json.loads(result)
    parsed_results = result.get("ParsedResults")[0]
    text_detected = parsed_results.get("ParsedText")
    #print(text_detected)
    text_dict[key] = text_detected
  #return  text_dict
getText_From_Image(image_dic)
print(text_dict)

{'Time series.PNG': 'Understanding Time Series\r\nData\r\nIn the previous chapter, we touched upon a general approach of time series analysis which\r\nconsists of two main steps:\r\n• Data visualization to check the presence of trend, seasonality, and cyclical\r\npatterns\r\n• Adjustment of trend and seasonality to generate stationary series\r\nGenerating stationary data is important for enhancing the time series forecasting model.\r\nDeduction of the trend, seasonal, and cyclical components would leave us with irregular\r\nfluctuations which cannot be modeled by using only the time index as an explanatory\r\nvariable. Therefore, in order to further improve forecasting, the irregular fluctuations are\r\nassumed to be independent and identically distributed (iid) observations and modeled by\r\na linear regression on variables other than the time index.\r\nFor example, house prices might exhibit both trend and seasonal (for example, quarterly)\r\nvariations. However, the residuals left a

In [8]:
text_dataframe = pd.DataFrame(list(text_dict.items()),columns = ['file_name','text'])
text_dataframe

,file_name,text
0,Time series.PNG,Understanding Time Series\r\nData\r\nIn the pr...
1,image data.PNG,Tensors - multidimensional arrays\r\nNow that ...
2,chat bots.PNG,CHAPTERI THE BELOVED CHATBOTS\r\nYou just type...
3,chat bot 2.PNG,Feature Releases and Bug Fixes\r\nLots of feat...
4,NLP 3.PNG,Natural Language Processing (NLP)\r\nNLP is an...
5,Chat Bods Intro.PNG,CHAPTER 1\r\nUsing Decision Trees in Chatbots\...
6,NLP2.PNG,1.1\r\nNatural language vs. programming langua...
7,NLP with Chotbot.PNG,Most chatbots will contain elements of all fiv...


Preprocessing


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = tfidf.fit_transform(text_dataframe['text'])
dtm

<8x130 sparse matrix of type '<class 'numpy.float64'>'
	with 326 stored elements in Compressed Sparse Row format>

Non-negative Matrix Factorization


In [10]:
from sklearn.decomposition import NMF
nmf_model = NMF(n_components=4,random_state=42)
nmf_model.fit(dtm)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=4, random_state=42, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

TASK: Print our the top 15 most common words for each of the 20 topics.

In [11]:
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['need', 'user', 'just', 'companies', 'facebook', 'document', 'lots', 'information', 'help', 'developers', 'demand', 'users', 'business', 'chatbot', 'chatbots']


THE TOP 15 WORDS FOR TOPIC #1
['require', 'based', 'huge', 'machine', 'chatbot', 'process', 'language', 'stages', 'data', 'text', 'learning', 'customer', 'pipeline', 'search', 'nlp']


THE TOP 15 WORDS FOR TOPIC #2
['better', 'chapter', 'different', 'll', 'called', 'forecasting', 'just', 'number', 'model', 'actually', 'data', 'images', 'look', 'time', 'series']


THE TOP 15 WORDS FOR TOPIC #3
['machine', 'understanding', 'involves', 'world', 'does', 'flows', 'pipeline', 'bit', 'll', 'use', 'programming', 'like', 'processing', 'language', 'natural']




In [12]:
topic_results = nmf_model.transform(dtm)
#topic_results.argmax(axis=1)

text_dataframe['Topic'] = topic_results.argmax(axis=1)

text_dataframe


,file_name,text,Topic
0,Time series.PNG,Understanding Time Series\r\nData\r\nIn the pr...,2
1,image data.PNG,Tensors - multidimensional arrays\r\nNow that ...,2
2,chat bots.PNG,CHAPTERI THE BELOVED CHATBOTS\r\nYou just type...,0
3,chat bot 2.PNG,Feature Releases and Bug Fixes\r\nLots of feat...,0
4,NLP 3.PNG,Natural Language Processing (NLP)\r\nNLP is an...,1
5,Chat Bods Intro.PNG,CHAPTER 1\r\nUsing Decision Trees in Chatbots\...,0
6,NLP2.PNG,1.1\r\nNatural language vs. programming langua...,3
7,NLP with Chotbot.PNG,Most chatbots will contain elements of all fiv...,1


In [13]:
cleanup_nums = {"Topic": {0: "Chat Bots", 1: "Time Series", 2:"Image Processing",3:"NLP"}}
text_dataframe.replace(cleanup_nums, inplace=True)
text_dataframe

,file_name,text,Topic
0,Time series.PNG,Understanding Time Series\r\nData\r\nIn the pr...,Image Processing
1,image data.PNG,Tensors - multidimensional arrays\r\nNow that ...,Image Processing
2,chat bots.PNG,CHAPTERI THE BELOVED CHATBOTS\r\nYou just type...,Chat Bots
3,chat bot 2.PNG,Feature Releases and Bug Fixes\r\nLots of feat...,Chat Bots
4,NLP 3.PNG,Natural Language Processing (NLP)\r\nNLP is an...,Time Series
5,Chat Bods Intro.PNG,CHAPTER 1\r\nUsing Decision Trees in Chatbots\...,Chat Bots
6,NLP2.PNG,1.1\r\nNatural language vs. programming langua...,NLP
7,NLP with Chotbot.PNG,Most chatbots will contain elements of all fiv...,Time Series
